In [52]:
import torch
import numpy as np
from utils import *

In [53]:
if torch.cuda.is_available():
  tensor = tensor.to('cuda')

In [112]:
def single_event_difference(i):
    event = importpbdatanumpy(i).T
    phi = torch.tensor(event[2])
    eta = torch.tensor(event[1])
    deta = eta - eta.unsqueeze(-1)
    dphi = phi - phi.unsqueeze(-1)
    ids = torch.arange(0,len(phi))
    mask = torch.ones_like(dphi).scatter_(1,ids.unsqueeze(1),0.)
    dphi = dphi[mask.bool()]
    deta = deta[mask.bool()]
    pimask = (dphi > torch.pi)*(-1) + (dphi < -torch.pi)*1
    dphi += pimask*2*torch.pi
    result = torch.stack([dphi,deta],dim=1)
    return result

In [113]:
for i in range(0,1000):
    single_event_difference(i)

In [114]:
single_event_difference(10000)

tensor([[-0.0830,  2.2600],
        [ 0.0480,  1.4460],
        [ 0.0680,  0.0790],
        ...,
        [-0.8470, -0.5080],
        [-1.5510,  3.9970],
        [-1.0840, -0.0770]], dtype=torch.float64)

In [115]:
event = importpbdatanumpy(0).T
phi = torch.tensor(event[2])
eta = torch.tensor(event[1])
deta = eta - eta.unsqueeze(-1)
dphi = phi - phi.unsqueeze(-1)
ids = torch.arange(0,len(phi))
mask = torch.ones_like(dphi).scatter_(1,ids.unsqueeze(1),0.)
dphi = dphi[mask.bool()]
deta = deta[mask.bool()]
pimask = (dphi > torch.pi)*(-1) + (dphi < -torch.pi)*1
dphi += pimask*2*torch.pi
result = torch.stack([dphi,deta],dim=1)